<a href="https://colab.research.google.com/github/sakamototaisei/python_colab/blob/main/%E3%83%9D%E3%83%BC%E3%83%88%E3%83%95%E3%82%A9%E3%83%AA%E3%82%AA%E6%A9%9F%E8%83%BD%E6%A0%AA%E4%BE%A1%E3%83%86%E3%82%AF%E3%83%8B%E3%82%AB%E3%83%AB%E8%A1%A8%E7%A4%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **株価のローソク足データを取得する**

In [6]:
pip install yahoo_finance_api2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3733 sha256=1b154c7edaa7481228e860342396c97daeeab98db2ba116b266a31fecbd4a50b
  Stored in directory: /root/.cache/pip/wheels/60/31/49/4f2dc65be0bb9c065b8f90701802babc434ad029dfaa9202df
Successfully built yahoo_finance_api2


In [7]:
import sys
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError
import pandas as pd

In [56]:
# SoftBank株価取得
my_share = share.Share('9984.T')
symbol_data = None
try:
    symbol_data = my_share.get_historical(
    share.PERIOD_TYPE_YEAR, 1,
    share.FREQUENCY_TYPE_DAY, 1)
except YahooFinanceError as e:
    print(e.message)
    sys.exit(1)
df = pd.DataFrame(symbol_data)
df["datetime"] = pd.to_datetime(df.timestamp, unit="ms")

In [57]:
df.head()

,timestamp,open,high,low,close,volume,datetime
0,1648425600000,5393.0,5421.0,5280.0,5402.0,12671800,2022-03-28
1,1648512000000,5488.0,5555.0,5433.0,5504.0,12102000,2022-03-29
2,1648598400000,5559.0,5633.0,5531.0,5632.0,15400800,2022-03-30
3,1648684800000,5595.0,5750.0,5542.0,5559.0,18054000,2022-03-31
4,1648771200000,5441.0,5609.0,5387.0,5578.0,15398600,2022-04-01


# **Plotlyでローソク足チャートと描く**

## インストール

In [19]:
pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

## plotly.graph_objects.Candlestick

In [58]:
fig = go.Figure(
    data=[go.Candlestick(x=df['datetime'], open=df['open'], high=df['high'], low=df['low'], close=df['close'], showlegend=False)]
)

# 下にあるバーを非表示にする場合
# fig.update(layout_xaxis_rangeslider_visible=False)

fig.show()

## タイトル・ラベルの追加、フォーマットの変更

In [59]:
fig.update_layout(
    title={
        'text': 'SoftBank(9984)の日足チャート',
        'y':0.9, # 縦軸でほぼ一番上表示
        'x':0.5, # 横軸でちょうど真ん中表示
    },
    yaxis_title='株価',
    xaxis_title='Date'
)

# 日付のフォーマット変更
fig.update_xaxes(tickformat='%Y/%m/%d')

# 株価のフォーマット変更(カンマ区切り)
fig.update_yaxes(separatethousands=True)

# 下にあるバーを非表示にする場合
# fig.update(layout_xaxis_rangeslider_visible=False)

fig.show()

## 営業日のみを表示する

特定のデータを非表示にする：randebreaks

不要な日付を抽出

In [60]:
#日付一覧を取得
d_all = pd.date_range(start=df['datetime'].iloc[0],end=df['datetime'].iloc[-1])
d_obs = [d.strftime("%Y-%m-%d") for d in df['datetime']]
d_breaks = [d for d in d_all.strftime("%Y-%m-%d").tolist() if not d in d_obs]

In [61]:
d_breaks[:10]

['2022-04-02',
 '2022-04-03',
 '2022-04-09',
 '2022-04-10',
 '2022-04-16',
 '2022-04-17',
 '2022-04-23',
 '2022-04-24',
 '2022-04-29',
 '2022-04-30']

In [63]:
fig.update_xaxes(rangebreaks=[dict(values=d_breaks)])
fig.show()

営業日のみを表示することができ、不自然な空白表示がなくなった

# **複数チャートを組み合わせる**

出来高や移動平均線、いろんなテクニカル指標を追加する

## plotly.subplot.make_subplotsで複数チャートを描く

基本的な使い方としては、subplotsでfigを定義し、fig.add_traceで各チャートにグラフを追加していく流れになる

## 出来高を追加する

make_subplotsで、rows（行）とcols（列）を指定し、shared_xaxes=Trueとしてx軸を共有するように設定します。

x_titleでxのラベル名を設定しています。

 

vertical_spacingはチャート間の隙間の大きさで0〜1で指定します。

この設定は好みですが、僕はだいたい0.05にすることが多いです。適宜調整してみてください。

 

row_widthではrow(行）の高さを指定します。x_titleはその名の通り、x軸名です。

y軸名は2つあるので、fig.update_yaxesを使ってそれぞれ指定します。

 

add_traceでrowとcolを指定することで、任意の場所にチャートを描くことができます。

1行目にローソク足チャート、2行目に出来高の棒グラフを描けばOKです。

In [68]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
 
# figを定義
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_width=[0.2, 0.7], x_title='Date')
 
# Candlestick 
fig.add_trace(
    go.Candlestick(x=df['datetime'], open=df['open'], high=df['high'], low=df['low'], close=df['close'], showlegend=False),
    row=1, col=1
)
 
# Volume
fig.add_trace(
    go.Bar(x=df['datetime'], y=df["volume"], showlegend=False),
    row=2, col=1
)
 
# Layout
fig.update_layout(
    title={
        'text': 'SoftBank(9984)の日足チャート',
        'y':0.9,
        'x':0.5,
    },
)

fig.update_xaxes(
    rangebreaks=[dict(values=d_breaks)], # 非営業日を非表示設定
    tickformat='%Y/%m/%d' # 日付のフォーマット変更
)
 
# ラベル名の設定とフォーマット変更（カンマ区切り）
fig.update_yaxes(separatethousands=True, title_text='株価', row=1, col=1) 
fig.update_yaxes(title_text='出来高', row=2, col=1)
 
fig.update(layout_xaxis_rangeslider_visible=False) #追加

fig.show()